In [ ]:
# import pandas and raw data
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Sherphie/rawdata/master/loans.csv')

In [ ]:
# 確認用
# ローデータの中身は？
df.head()

In [ ]:
# 確認用
# ローデータの形は？　(m行,n列)
df.shape

In [ ]:
# 特徴値(feature)のXと目標値(target)のyをローデータから分割
X = df.drop('safe_loans', axis=1)
y = df.safe_loans

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X.head()

In [ ]:
# sklearn初期化
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

# ローデータはcategorical形なので数値に転換
d = defaultdict(LabelEncoder)
X_trans = X.apply(lambda x: d[x.name].fit_transform(x))

X_trans.head()

In [ ]:
# 交差検証のため、ローデータを訓練用と検証用の　「train」「test」の　二種類に分割
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_trans, y, random_state=1)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
# データ用意完成、訓練用の「train」データつかって決定木生成
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(X_train, y_train)

In [ ]:
# 生成した決定木を可視化する
with open("safe-loans.dot", 'w') as f:
     f = tree.export_graphviz(clf,
                              out_file=f,
                              max_depth = 3,
                              impurity = True,
                              feature_names = list(X_train),
                              class_names = ['not safe', 'safe'],
                              rounded = True,
                              filled= True )

In [ ]:
# 生成した決定木データ.dotをグラフの.pngに転換
from subprocess import check_call
check_call(['C:/Program Files (x86)/Graphviz2.38/bin/dot','-Tpng','safe-loans.dot','-o','safe-loans.png'])

In [ ]:
#　グラフ出力
from IPython.display import Image as PImage
from PIL import Image, ImageDraw, ImageFont
img = Image.open("safe-loans.png")
draw = ImageDraw.Draw(img)
img.save('output.png')
PImage("output.png")

In [ ]:
# 手動ランダム検証？　Xから適当にサンプル1本つかって先ほど生成した決定木を検証
test_rec = X_test.iloc[1,:]
clf.predict([test_rec])

In [ ]:
# 目標値は？　yから答えを
y_test.iloc[1]

In [ ]:
# 交差検証　→　「test」データつかって決定木の判断正解率を評価する
from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf.predict(X_test))

In [ ]:
# 約6割　QAQ